# Scrap data from __[FBref.com](https://fbref.com/en/)__
Tutorial loosely followed: [Tutorial](https://medium.com/analytics-vidhya/intro-to-scraping-basketball-reference-data-8adcaa79664a)

In [ ]:
!pip install cchardet

Necassary Imports

In [ ]:
import numpy as np  #load up the libraries and object definitions we need
import pandas as pd
from pandas import DataFrame, Series
import io
from string import ascii_lowercase
import lxml
import requests

import cchardet


# load up my visualization system, and call the object plt
import matplotlib.pyplot as plt

# tell ipython notebook to print visualizations inline
%pylab
%matplotlib inline

from urllib.request import urlopen
from bs4 import BeautifulSoup, SoupStrainer

#### Beautiful Soup is used for all scraping
For each season, Fb ref has a page for all players who in the top leagues. For each year, go to the correct url and get all anchor tags that has a href that matches the regular expression for a player's page. All get each player's nation and birth year

In [ ]:
import re
players = []
playersdf = pd.DataFrame()
years = ["2010-2011", "2011-2012","2012-2013","2013-2014","2014-2015","2015-2016","2016-2017","2017-2018","2018-2019","2019-2020","2020-2021","2021-2022","2022-2023"]
for y in years:
  #for c in pages:
  product = SoupStrainer('div',{'id': 'div_stats_standard'})
  print(product)
 
  html = urlopen("https://fbref.com/en/comps/Big5/" + y + "/stats/players/" + y + "-Big-5-European-Leagues-Stats")
  
    #url4 = "https://fbref.com/en/players/" + c + "/"
    #print(url4)
    # collect HTML data
  #html = urlopen(url4)
  soup2 = BeautifulSoup(html, 'lxml', parse_only=product)
  #soup2 = soup.find(id="div_stats_standard")
  #soup2
  headers = [th.getText() for th in soup2.findAll('tr', limit=2)[1].findAll('th')]
  headers.pop(0)

  rows = soup2.findAll('tr')[2:]
  rows_data = [[td.getText() for td in rows[i].findAll('td')]
                      for i in range(len(rows))] 

  # print(headers)
  # print(rows_data) 

  df = pd.DataFrame(rows_data, columns = headers)
  dffinal = df[['Nation', 'Born']]
  # dffinal = dffinal.iloc[:, :-1]
  #dffinal.drop(dffinal.columns[len(dffinal.columns)-1], axis=1, inplace=True)
  df2 = dffinal[dffinal.Born.notnull()]
  #df2 = df2.fillna('na Na')
  df2.replace(to_replace=[""], value='na Na', inplace=True)
  df2.replace(to_replace=[" SCG"], value='na SCG', inplace=True)
  df2.replace(to_replace=[" MCO"], value='na MCO', inplace=True)
  if y == '2012-2013':
    df2.drop(df2.tail(1).index,inplace=True)
  if y == '2013-2014':
    df2.drop(df2.tail(3).index,inplace=True)
  if y == '2014-2015':
    df2.drop(df2.tail(3).index,inplace=True)
  if y == '2016-2017':
    df2.drop(df2.tail(2).index,inplace=True)
  if y == '2017-2018':
    df2.drop(df2.tail(1).index,inplace=True)
  #print(df2['Nation'])
  df2['Nation'] = [x.split()[1] for x in df2['Nation']]
  #df2 = df2[df2.Age != '']
    # create beautiful soup object from HTML
  #soup = BeautifulSoup(html, 'html.parser')
    #uncomment to see HTML
  #soup2 = soup.find(class_="section_wrapper")
  print(y)
  players = []
  for link in soup2.find_all('a'):
      #html2 = urlopen("https://fbref.com" + link.get('href'))
      #soup2 = BeautifulSoup(html2, 'html.parser')
      #if(soup2.findAll(text='var sr_gender = "men";') != []):
      if(re.search(r"[a-z]{7}[/]{1}[a-h0-9]{8}[\/]{1}[a-zA-Z-]{2,50}$",link.get('href')) != None):
        players.append("https://fbref.com" + link.get('href'))
        # nameInt = x.rfind('/')
        # name = x[(nameInt+1):]
        # id = x[(nameInt-8):nameInt]
        # players.append(x)
  
  # for p in players:
  #   print(p)
  df2['id'] = players
  playersdf = playersdf.append(df2)


In [ ]:
playersdf.replace(to_replace=[" MCO"], value='na MCO', inplace=True)
#playersdf['Nation'] = [x.split()[1] for x in playersdf['Nation']]
playersdf

Remove all duplicate players who had played more than 1 year

In [ ]:
df2 = playersdf.drop_duplicates()
df2

In [ ]:
playersList = df2['id'].tolist()
playersBirth = df2['Born'].tolist()
playersNation = df2['Nation'].tolist()

Go to each players page, get their domestic stats, 1 row for each season they have played. Append to a dataframe

In [ ]:
import warnings
with warnings.catch_warnings():
    warnings.simplefilter('ignore')
    i = 0
    playersdf = pd.DataFrame()
    for x in playersList:
      #print(x)
      # html = urlopen(x)
      # soup = BeautifulSoup(html, features="lxml")
      # soup1 = soup.find(id="div_stats_standard_dom_lg")

      product = SoupStrainer('div',{'id': 'div_stats_standard_dom_lg'})
      #print(product)
      soup1 = BeautifulSoup(requests.get(x).content,'lxml', parse_only=product)

      
        #print(soup1)
      if(soup1 != None):# and (soup.findAll(text='var sr_gender = "men";') != [])):
          #print(soup1)
          #print("Hi")
        headers = [th.getText() for th in soup1.findAll('tr', limit=2)[1].findAll('th')]
        headers
        headers.pop(0)
        #gets name of player from URL
        nameInt = x.rfind('/')
        name = x[(nameInt+1):]
        id = x[(nameInt-8):nameInt]
        #print(name)
        # get rows from table
        rows = soup1.findAll('tr')[2:]
        rows_data = [[td.getText() for td in rows[i].findAll('td')]
                            for i in range(len(rows))]

        #print(rows_data)
        # if you print row_data here you'll see an empty row
        # so, remove the empty row
        season = soup1.findAll('tr')[2:]
        season = [[th.getText() for th in rows[i].findAll('th')]
                            for i in range(len(rows))]
        #rows_data.pop(20)
        #div_stats_standard_dom_lg
        # for simplicity subset the data for only 39 seasons
        #rows_data = rows_data[0:38]


        #print(season)
        s1 = []
        s2 = []
        for s in season:
          s1.append(s[0])
          s2.append(re.search(r"^[0-9-]{4,9}$",s[0]) != None)

        #for s in s1:
          #print(re.search(r"^[0-9-]{4,9}$",s) != None)
          
          
        df = pd.DataFrame(rows_data, columns = headers)
        #df = pd.DataFrame(rows_data)
        df['Season'] = s1
        df['tf'] = s2
        df['Name'] = name
        df['ID'] = id
        df['Nation'] = playersNation[i]
        df['Birth Year'] = playersBirth[i]
        #indexNames = df[ re.search(r"^[0-9-]{4,9}$",df['seasons']) != None ].index
        #df.drop(indexNames , inplace=True)
        #print(name)
        dfnew = df.loc[df['tf'] == True]
        dffinal = dfnew[['Season', 'Squad', 'Comp', 'Country','Name', 'ID', 'Birth Year', 'Nation', 'LgRank' ,'MP','Gls']]
        dffinal = dffinal.iloc[:, :-1]
        #df5 = dffinal.assign(Name=id)
        # dffinal['ID']= id
        #dffinal
        playersdf = playersdf.append(dffinal)

        print(i)
        i += 1


In [ ]:
playersdf.reset_index(drop=True, inplace=True)

export file

In [ ]:
playersdf.to_csv('playersHistory.csv', index=False)